In [265]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import os, json
from os.path import dirname
from pandas import DataFrame, concat
from tqdm import tqdm
import seaborn as sns 

import re


In [266]:
dir = '../data/data_july_24/'
interim_files = glob('{}*interim'.format(dir))
all_files = glob("{}*".format(dir))
zip_files = glob("{}*.zip".format(dir))



In [267]:
full_files = [item for item in all_files if ((item not in interim_files) and (item not in zip_files))]


In [268]:
full_files[13]

user_id = r's\d{1,3}'
task = r'[\d]_([a-z]+_?[a-z]+)'
date = r'(20\d*)'

In [269]:
cols = ['filename','user_id','task','date','full','mb']
df = pd.DataFrame(columns=cols)

In [270]:
for file in full_files:

    tmp_user = re.search(user_id,file)
    tmp_task = re.search(task, file)
    tmp_data = re.search(date, file)

    if not tmp_user:
        print(file, tmp_user)

    if not tmp_task:
        print(file, tmp_task)

    if not tmp_data:
        print(file, tmp_data)

    df_tmp =  pd.DataFrame(np.array([file,tmp_user[0][1:], tmp_task[0][2:], tmp_data[0], 'full', os.path.getsize(file)/1024]).reshape(1,6),
                           columns=cols)
    df = pd.concat([df,df_tmp])

In [271]:
df.groupby(['task']).nunique()['filename']

task
mrst               30
pgng               24
scream_acq         25
scream_recovery    23
twostep            26
Name: filename, dtype: int64

In [272]:
import re
from datetime import datetime

def convert_to_standard_datetime(datetime_str):
    # Define regex patterns to match different datetime formats
    patterns = [
        r'(\d{4})(\d{1})(\d{1})(\d{1})(\d{1})(\d{1})',   # yyyymdhms (e.g., 20243110101)
        r'(\d{4})(\d{1})(\d{1})(\d{2})(\d{2})(\d{2})',   # yyyymddhms (e.g., 202471610101)
        r'(\d{4})(\d{2})(\d{1})(\d{1})(\d{1})(\d{1})',   # yyyymmddhms (e.g., 202407260101)
        r'(\d{4})(\d{2})(\d{2})(\d{2})(\d{2})(\d{2})',   # yyyymmddhhmmss (e.g., 20240726101010)
        r'(\d{4})(\d{1})(\d{2})(\d{1})(\d{2})(\d{2})',   # yyyymddhhmmss (e.g., 202472601010)
        r'(\d{4})(\d{2})(\d{1})(\d{2})(\d{2})(\d{2})',   # yyyymmddhhmmss (e.g., 202407260101)
        r'(\d{4})(\d{2})(\d{2})(\d{1})(\d{2})(\d{2})',   # yyyymmddhhmmss (e.g., 202407261010)
    ]
    
    for pattern in patterns:
        match = re.match(pattern, datetime_str)
        if match:
            year, month, day, hour, minute, second = match.groups()
            # Pad month, day, hour, minute, and second with leading zeros if necessary
            month = month.zfill(2)
            day = day.zfill(2)
            hour = hour.zfill(2)
            minute = minute.zfill(2)
            second = second.zfill(2)
            return f"{year}-{month}-{day} {hour}:{minute}:{second}"
    
    return "Invalid datetime format"

# # Examples
# datetimes = ["20243110101", "202471610101", "202407260101", "20240726101010", "202472601010", "202407260101", "202407261010"]
# converted_datetimes = [convert_to_standard_datetime(datetime_str) for datetime_str in datetimes]
# print(converted_datetimes)


In [273]:
df['standard_datetime'] = df.date.apply(lambda x:convert_to_standard_datetime(x))

In [274]:
df.mb = df.mb.astype(float)


In [275]:
df.groupby(['user_id']).nunique().reset_index()[['user_id','task']].groupby('task').nunique()

,user_id
task,
1,3
2,3
3,2
4,8
5,15


In [276]:
## all users

particiapnts_df = pd.read_csv('participants.csv', sep=",")  
valid_participants = particiapnts_df[particiapnts_df['VALID']==True]



In [277]:
valid_participants['mrst'] = valid_participants[['RISK_1','RISK_2']].apply(lambda x: x[0] and x[1], axis=1)
valid_participants['pgng'] = valid_participants[['PGNG_1','PGNG_2']].apply(lambda x: x[0] and x[1], axis=1)
valid_participants['twostep'] = valid_participants[['2STEP_1','2STEP_2']].apply(lambda x: x[0] and x[1], axis=1)
valid_participants['scream_acq'] = valid_participants['SCREAM_1'] 
valid_participants['scream_recovery'] = valid_participants['SCREAM_2'] 



/var/folders/f7/jcqxh9dj3tv469cncv9966q40000gn/T/ipykernel_11312/3533921764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_participants['mrst'] = valid_participants[['RISK_1','RISK_2']].apply(lambda x: x[0] and x[1], axis=1)
/var/folders/f7/jcqxh9dj3tv469cncv9966q40000gn/T/ipykernel_11312/3533921764.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_participants['pgng'] = valid_participants[['PGNG_1','PGNG_2']].apply(lambda x: x[0] and x[1], axis=1)
/var/folders/f7/jcqxh9dj3tv469cncv9966q4

In [278]:
#valid_participants[['RISK','SCREAM_1','SCREAM_2,'PGNG','2STEP','VALID']]

In [279]:
df.date = df.date.astype(float)

In [280]:
df_pivot = df.pivot_table(index='user_id', columns='task', values='date').reset_index()
df_pivot[['mrst','pgng','scream_acq','scream_recovery','twostep']] = df.pivot_table(index='user_id', columns='task', values='date').reset_index()[['mrst','pgng','scream_acq','scream_recovery','twostep']].notna()



In [281]:
df_pivot['user_id'] = df_pivot.user_id.astype(int)

In [282]:
# valid_participants[['participant_id','mrst','pgng','scream_acq','scream_recovery','twostep']]

In [283]:
merge_df = pd.merge(df_pivot, valid_participants[['participant_id','mrst','pgng','scream_acq','scream_recovery','twostep']], left_on='user_id', right_on='participant_id')

In [284]:
# merge_df

In [285]:
merge_df['mrst_check'] = merge_df[['mrst_x','mrst_y']].apply(lambda x: False if x[1] and not x[0] else True, axis=1)
merge_df['pgng_check'] = merge_df[['pgng_x','pgng_y']].apply(lambda x: False if x[1] and not x[0] else True, axis=1)
merge_df['twostep_check'] = merge_df[['twostep_x','twostep_y']].apply(lambda x: False if x[1] and not x[0] else True, axis=1)
merge_df['scream_acq_check'] = merge_df[['scream_acq_x','scream_acq_y']].apply(lambda x: False if x[1] and not x[0] else True, axis=1)
merge_df['scream_rec_check'] = merge_df[['scream_recovery_x','scream_recovery_y']].apply(lambda x: False if x[1] and not x[0] else True, axis=1)


In [287]:
# merge_df[['user_id','mrst_check','pgng_check','twostep_check','scream_acq_check','scream_rec_check']].drop_duplicates().groupby('user_id').sum()

In [288]:
#missing files
missing_check = []
names = ['mrst_check','pgng_check','twostep_check','scream_acq_check','scream_rec_check']
for row in merge_df[['user_id','mrst_check','pgng_check','twostep_check','scream_acq_check','scream_rec_check']].drop_duplicates().groupby('user_id').sum().iterrows():
    
    user = row[0]
    if len(row[1][row[1]==0].reset_index()['index'])>0:
        missing_check.append((user, row[1][row[1]==0].reset_index()['index']))

In [301]:
missing_check[7]

(131,
 0       twostep_check
 1    scream_rec_check
 Name: index, dtype: object)

In [295]:
df[df.user_id=='108']

,filename,user_id,task,date,full,mb,standard_datetime
0,../data/data_july_24/s108_pgng_202362115524,108,pgng,2.023621e+11,full,169.056641,2023-06-02 01:01:05
0,../data/data_july_24/s108_mrst_2023621162420,108,mrst,2.023621e+12,full,146.208984,2023-06-02 01:01:06


In [238]:
import os


In [297]:
dir = '../data/incomplete_search/'
files_incom = glob('{}*'.format(dir))

for file in files_incom:
    # date = re.search('2023-07-04',file)

    # if date:
    # print(file)
    f = open(file)
    data = json.load(f)

    if len(data)>1:
        if data[0]['sub'] == 110:
            print(file, data[0]['sub'], data[0]['task'])
                  
                  #'../data/data_july_24/'+data[0]['fout']+'.json')

            # os.rename(file, '../data/data_july_24/'+data[0]['fout']+'.json')

    

In [239]:
file

'../data/incomplete_search/2024-04-07 08:20:45'

In [243]:
'../data/data_july_24/'+data[0]['fout']+'.json'

'../data/data_july_24/s999_scream_acq_ext_202382181824.json'